In [1]:
%autosave 0

Autosave disabled


In [2]:
import sys
import math
import random
from operator import itemgetter

In [3]:
random.seed(0)

In [ ]:
class ItemBasedCF():
    def __init__(self):
        self.trainset = {}
        self.testset = {}
        
        self.n_sim_movie = 20
        self.n_rec_movie = 10
        
        self.movie_sim_mat = {}
        self.movie_popular = {}
        self.movie_count = 0
        
        print('Similar movie number = %d' % self.n_sim_movie) 
        print('Recommended movie number = %d' % self.n_rec_movie)
        
    @staticmethod
    def loadfile(filename):
        fp = open(filename, 'r')
        for i, line in enumerate(fp):
            yield line.strip('\r\n')
            if i > 0 and i % 100000 == 0:
                print('loading %s(%s)' %(filename, i))
        fp.close()
        print('load %s success' % filename)
        
    def generate_dataset(self, filename, pivot=0.7):
        trainset_len = 0
        testset_len = 0
        
        for line in self.loadfile(filename):
            user, movie, rating, _ = line.split('\t')
            if random.random < pivot:
                self.trainset.setdefault(user, {})
                self.trainset[user][movie] = int(rating)
                trainset_len += 1
            else:
                self.testset.setdefault(user, {})
                self.testset[user][movie] = int(rating)
                testset_len += 1
                
        print('分离训练集和测试集成功')
        print('train set = %s' % trainset_len)
        print('test set = %s' % testset_len)
        
    def cal_movie_sim(self):
        print('counting movies number and popularity ...')
        for _, movies in self.trainset.items():
            for movie in movies:
                if movie not in self.movie_popular:
                    self.movie_popular[movie] = 0
                self.movie_popular[movie] += 1
        print('count movies number and popularity success')
        
        self.movie_count = len(self.movie_popular)
        print('total movie number = %d' % self.movie_count)
        
        itemsim_mat = self.movie_sim_mat
        print('building co-rated user matrix ...')
        for _, movies in self.trainset.items():
            for m1 in movies:
                for m2 in movies:
                    if m1 == m2:
                        continue
                    itemsim_mat.setdefault(m1, {})
                    itemsim_mat[m1].setdefault(m2, 0)
                    itemsim_mat[m1][m2] += 1
        print('build co-rated users matrix success')
        
        print('calculating movie similarity matrix ...')
        simfactor_count = 0
        PRINT_STEP = 2000000
        for m1, related_movies in itemsim_mat.items():
            for m2, count in related_movies.items():
                itemsim_mat[m1][m2] = count / math.sqrt(self.movie_popular[m1] * self.movie_popular[m2])
                simfactor_count += 1
                if simfactor_count % PRINT_STEP == 0:
                    print('calculating movie similarity factor(%d)' % simfactor_count)
        print('calculate movie similarity matrix(similarity factor) success')
        print('Total similarity factor number = %d' % simfactor_count)
        
    def recommend(self, user):
        K = self.n_sim_movie
        N = self.n_rec_movie
        rank = {}
        watched_movies = self.trainset[user]
        
        for movie, rating in watched_movies.items():
            for related_movie, w in sorted(self.movie_sim_mat[movie].items(),
                                          key=itemgetter(1),
                                          reverse=True)[0:K]:
                if related_movie in watched_movies:
                    continue
                rank.setdefault(related_movie, 0)
                rank[related_movie] += w * rating
        return sorted(rank.items(),
                     key=itemgetter(1),
                     reverse=True)[0:N]
    
    def evaluate(self):
        print('Evaluation start ...')
        
        N = self.n_rec_movie
        hit = 0
        rec_count = 0
        test_count = 0
        all_rec_movies = set()
        popular_sum = 0
        
        for i, user in enumerate(self.trainset):
            if i > 0 and i % 500 == 0:
                print('recommended for %d users' % i)
            test_movies = self.testset.get(user, {})
            rec_movies = self.recommend(user)
            
            for movie, _ in rec_movies:
                if movie in test_movies:
                    hit += 1
                all_rec_movies.add(movie)
                popular_sum += math.log(1 + self.movie_popular[movie])
            rec_count += 1
            test_count += len(test_movies)
            
        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        coverage = len(all_rec_movies) / (1.0 * self.movies_count)
        popularity = popular_sum / (1.0 * rec_count)
        
        print('precision = %.4f \t recall=%.4f \t coverage=%.4f \t popularity=%.4f' % (precision, recall, coverage, popularity))
        

In [ ]:
ratingfile = 'u.data'
 # 创建ItemCF对象
itemcf = ItemBasedCF()
# 将数据按照 7:3的比例，拆分成：训练集和测试集，存储在usercf的trainset和testset中
itemcf.generate_dataset(ratingfile, pivot=0.7)
# 计算用户之间的相似度
itemcf.calc_movie_sim()
# 评估推荐效果
# itemcf.evaluate()
# 查看推荐结果用户
user = "2"
print("推荐结果", itemcf.recommend(user))
print("---", itemcf.testset.get(user, {}))